In [ ]:
import datetime
import pickle
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder

df = pd.read_csv('D:\Persistent_Work\Assignment\Car details.csv')
class DataCleaning:

    def __init__(self):
        # self.df = None
        self.onehot_encoder = None
        self.label_encoder = None
        self.imputed_data = None
        self.imputed_array = None
        self.label = None
        self.features = None
        
        print("csv read")

    def feature_splitting(self, data):
        # df = pd.read_csv('D:\Persistent_Work\Assignment\Car details.csv')
        """ Docstring
        """
        name_feature_split = df["name"].str.split(" ", expand = True, n = 1)
        df['manufacturer'] = name_feature_split[0]
        # print("feature_splitting")
        return df['manufacturer']
        
        # print("unique values present inside the features :", name_df_1[0].nunique())
        # name_df_1[0].value_counts(normalize=True)*100

    def feature_creation_car_age(self,df):
        today = datetime.date.today() 
        yr = today.year

        df['car_age'] = yr - df['year']
        print("feature_creation_car_age execute")
        return df['car_age']
    
    def feature_extraction_mileage(self,df):
        corrected_mileage = []
        for i in df.mileage:
            if str(i).endswith('km/kg'):
                i = i[:-6]
                i = float(i) * 1.40    # 1.40 is unit conversion value for km/kg to kmpl
                corrected_mileage.append(float(i))

            elif str(i).endswith('kmpl'):
                i = i[:-6]
                corrected_mileage.append(float(i))
        df['corrected_mileage_kmpl'] = pd.Series(corrected_mileage) 
        print("feature_extraction_mileage executed")  
        return df['corrected_mileage_kmpl']

    def feature_extraction_engine_size(self,df):
        df["engine_size_cc"] = df["engine"].str.split(" ", expand = True, n =1)[0]
        df["engine_size_cc"] = df["engine_size_cc"].astype("float")
        print("feature_extraction_engine_size executed") 
        return df["engine_size_cc"]
    
    def feature_extraction_maximum_power(self,df):
        df['max_power_bhp'] = df['max_power'].str.replace(' bhp','')
        df['max_power_bhp'] = pd.to_numeric(df['max_power_bhp'])
        print("feature_extraction_maximum_power executed") 
        return df['max_power_bhp']

    def drop_irrelevent_feature(self, df, feature_list):
        """
            Method Name: drop_irrelevent_feature
            Description: This method drops the unwanted columns

            :param data: A pandas dataframe
            :param column_names: List of columns that is required to drop
            :return: pandas dataframe with the specified list of columns removed
        """
        df = df.drop(feature_list, axis=1)     # feature_list = ["name", mileage, torque,engine, max_power]
        print("drop_irrelevent_feature executed")
        return df

    def rename_features(self,df):

        df = df.rename({"fuel" : "fuel_type","seats" : "no_seats"}, inplace = True, axis = 1)
        print("rename_features executed")
        return df
    
    def missing_values_imputation(self, df):
        """
            Method Name: impute_missing_values
            Description: This method replaces all the missing values in the dataframe using median

        :param data: A pandas dataframe
        :return: A dataframe which has all the missing values imputed
        """
        missing_value_features = df.iloc[: , list(np.where(df.isna().sum() > 0)[0]) ].columns
        for i in missing_value_features:
            if i == 'corrected_mileage_kmpl':
                df['corrected_mileage_kmpl'].fillna(df['corrected_mileage_kmpl'].mean(), inplace = True)
            elif i == 'engine_size_cc':
                df['engine_size_cc'].fillna(df['engine_size_cc'].median(), inplace = True)
            elif i == 'max_power_bhp':
                df['max_power_bhp'].fillna(df['max_power_bhp'].mean(), inplace = True)
            elif i == "no_seats":
                df['no_seats'].fillna(df['no_seats'].mode()[0], inplace = True)
        print("missing_values_imputation executed")
        
        return df
                
            
        
        # try:
        #     numeric_data = [column for column in data.select_dtypes(['int', 'float'])]
        #     for col in numeric_data:
        #         data[col].fillna(data[col].median(), inplace=True)
        #     return data

        # except Exception as e:
            
        #     raise Exception()

    def feature_encoding(self, df):
        """
            
        """
        # get_dummies - OneHotEncoding >> 'fuel_type','seller_type'
        df = pd.get_dummies(df, columns = ['fuel_type','seller_type'],drop_first= True, prefix = None)

        #OneHotEncoding >> "manufacturer"
        self.onehot_encoder = OneHotEncoder()
        encode_manufacturer = self.onehot_encoder.fit_transform(df[['manufacturer']])
        manfacturer_df = pd.DataFrame(encode_manufacturer.toarray(),columns = df['manufacturer'].unique().tolist())
        df = df.join(manfacturer_df, how='outer')
        
        with open('onehot_encode_manfact.pkl', 'wb') as f:     # save in current directory onehot of manfacturer
            pickle.dump(encode_manufacturer, f)

        #LabelEncoding >>  "owner"
        self.label_encoder = LabelEncoder()
        label_encode_transmission = self.label_encoder.fit_transform(df["transmission"])
        df["transmission"] = label_encode_transmission

        with open('label_encode_transmission.pkl', 'wb') as f:
            pickle.dump(label_encode_transmission, f)

        #LabelEncoding >>  "owner"
        label_encode_owner = self.label_encoder.fit_transform(df["owner"])
        df["owner"] = label_encode_owner
        
        with open('label_encode_owner.pkl', 'wb') as f:
            pickle.dump(label_encode_owner, f)
        
        print("feature_encoding executed")
        return df


    
    # def get_all_methods(self,df):
    #     self.feature_extraction_engine_size(df)
    #     self.feature_extraction_maximum_power(df)
    #     self.drop_irrelevent_feature(df, ["name", 'mileage', 'torque','engine', 'max_power'])
    #     self.rename_features(df)
    #     self.missing_values_imputation(df)
    #     self.feature_encoding(df)



# model_instance = DataCleaning()
# model_instance.feature_splitting(df)
# model_instance.feature_creation_car_age(df)
# model_instance.feature_extraction_mileage(df)

# model_instance.get_all_methods(df)

